In [2]:
embeding_matrix = load_pickle_file("./embeding_matrix")
word2index = load_pickle_file("./word2index_titles")
index_sequences_titles = load_pickle_file("./index_sequences_titles.pkl")
index_sequences_text = load_pickle_file("../index_sequences.pkl")
labels = load_pickle_file("../labels")
SEQUENCE_LENGTH = 618
SEQUENCE_LENGTH_TITLE = 10
VOCAB_SIZE = len(embeding_matrix)
EMB_DIMENSION = 300
VALIDATION_SPLIT = 0.1
PAD_SEQ_VALUE = 3001066

In [3]:
import numpy as np
same_lenght_seq = [seq[:SEQUENCE_LENGTH].reshape(1,SEQUENCE_LENGTH) if len(seq) >= SEQUENCE_LENGTH 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH) 
                   for seq in index_sequences_text]
same_lenght_seq = np.concatenate(same_lenght_seq,axis=0)

same_lenght_titles = [seq[:SEQUENCE_LENGTH_TITLE].reshape(1,SEQUENCE_LENGTH_TITLE) if len(seq) >= SEQUENCE_LENGTH_TITLE 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH_TITLE - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH_TITLE) 
                   for seq in index_sequences_titles]
same_lenght_titles = np.concatenate(same_lenght_titles,axis=0)
same_lenght_titles = np.array([[int(x) for x in l] for l in same_lenght_titles])


In [4]:
import random
test_size = int(len(labels) * 0.2)
true_labels = np.where(np.array(labels)==0)[0]
start_index = true_labels[0]
random_test_true_indexes = [int(x * len(true_labels)+start_index) for x in np.random.random(test_size//2)]
random_test_false_indexes = [int(x * (len(labels) - len(true_labels))) for x in np.random.random(test_size//2)]
x_test_indexes = random_test_true_indexes + random_test_false_indexes
random.shuffle(x_test_indexes)

x_train_indexes = [x for x in range(0,len(labels)) if x not in x_test_indexes]

random.shuffle(x_train_indexes)


labels = np.vstack(np.array([1,0]).reshape(1,2) if x == 0 else np.array([0,1]).reshape(1,2) for x in labels)

y_test = np.array(labels)[x_test_indexes]
y_train = np.array(labels)[x_train_indexes]

x_train_text = same_lenght_seq[x_train_indexes]
x_train_title = same_lenght_titles[x_train_indexes]

x_test_text = same_lenght_seq[x_test_indexes]
x_test_title = same_lenght_titles[x_test_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  from ipykernel import kernelapp as app


Dataset split

In [5]:
from sklearn.model_selection import train_test_split

# x_train_indexes, x_test_indexes, y_train, y_test = train_test_split(list(range(len(same_lenght_seq))), 
#                                                                                     labels, test_size=0.2,
#                                                                                     random_state=42,
#                                                                                     shuffle = True)


In [5]:
#create validation dataset
data_size = len(x_train_text)
perm = np.random.permutation(data_size)
idx_train = perm[:int(data_size*(1-VALIDATION_SPLIT))]
idx_val = perm[int(data_size*(1-VALIDATION_SPLIT)):]

data_train_text = x_train_text[idx_train]
data_train_title = x_train_title[idx_train]

labels_train = y_train[idx_train]

data_val_text = x_train_text[idx_val]
data_val_title = x_train_title[idx_val]

labels_val = y_train[idx_val]

In [13]:
data_train.shape

(7380, 434)

<h1>Convolutional neural networks</h1>

In [6]:
import tensorflow as tf
from tensorflow import keras as tk
from tensorflow.keras import layers as tfkl
from sklearn.metrics import mean_squared_error,mean_absolute_error

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Conv1D, MaxPool1D, SpatialDropout1D, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [7]:
model_checkpoint = ModelCheckpoint("./Checkpoints", save_best_only=True, save_weights_only=True)


In [36]:
data_train_text.shape

(11397, 618)

In [7]:
#Text model
LSTM_HIDDEN_DIM_SIZE = 10
NUM_FILTERS = 25
KERNEL_SIZE = 18
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15
input_1 = Input(shape=(618,))
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH,
        trainable=False)(input_1)
x1 = Conv1D(NUM_FILTERS, KERNEL_SIZE, activation = 'relu',kernel_regularizer =tf.keras.regularizers.l2(l=0.01))(embedding_layer)
x1 = Dropout(0.2)(x1)
x1 = MaxPool1D(pool_size = 12)(x1)
x1 = LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm)(embedding_layer)

#Titles model
NUM_FILTERS = 2
KERNEL_SIZE = 4
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15

input_2 = Input(shape=(10,))
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH_TITLE,
        trainable=False)(input_2)


x2 = Conv1D(NUM_FILTERS, KERNEL_SIZE, activation = 'relu',kernel_regularizer =tf.keras.regularizers.l2(l=0.01))(embedding_layer)
x2 = Dropout(0.2)(x2)
x2 = MaxPool1D(pool_size = 2)(x2)
x2 = GlobalMaxPool1D()(x2)

concat = tk.layers.Concatenate()([x1,x2])
output = Dense(2,activation='sigmoid')(concat)
full_model = tf.keras.Model(inputs=[input_1, input_2], outputs=[output])

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
full_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 10, 300)      900320100   input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 7, 2)         2402        embedding_1[0][0]                
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 618)]        0                                            
______________________________________________________________________________________________

In [23]:
full_model.reset_states()

In [10]:
opt = tk.optimizers.Adam(learning_rate=0.02)

In [11]:
full_model.compile(loss='binary_crossentropy',
        optimizer=opt,
        metrics=['acc'])
history2 = full_model.fit([data_train_text,data_train_title],labels_train,epochs = 20,batch_size = 256, 
                     validation_data = [[data_val_text,data_val_title],labels_val],shuffle=True)

Train on 11706 samples, validate on 1301 samples
Epoch 1/20
11706/11706 [==============================] - 143s 12ms/sample - loss: 0.5861 - acc: 0.7289 - val_loss: 0.5728 - val_acc: 0.7337
Epoch 2/20
11706/11706 [==============================] - 140s 12ms/sample - loss: 0.5612 - acc: 0.7459 - val_loss: 0.5421 - val_acc: 0.7683
Epoch 3/20
11706/11706 [==============================] - 140s 12ms/sample - loss: 0.5441 - acc: 0.7559 - val_loss: 0.5402 - val_acc: 0.7683
Epoch 4/20
11706/11706 [==============================] - 136s 12ms/sample - loss: 0.5261 - acc: 0.7700 - val_loss: 0.4974 - val_acc: 0.7902
Epoch 5/20
11706/11706 [==============================] - 136s 12ms/sample - loss: 0.5717 - acc: 0.7554 - val_loss: 0.5612 - val_acc: 0.7567
Epoch 6/20
11706/11706 [==============================] - 131s 11ms/sample - loss: 0.5508 - acc: 0.7635 - val_loss: 0.5514 - val_acc: 0.7725
Epoch 7/20
11706/11706 [==============================] - 129s 11ms/sample - loss: 0.5511 - acc: 0.7745 -

In [16]:
history2 = full_model.fit([data_train_text,data_train_title],labels_train,epochs = 10,batch_size = 256, 
                     validation_data = [[data_val_text,data_val_title],labels_val],shuffle=True)

Train on 11706 samples, validate on 1301 samples
Epoch 1/10
11706/11706 [==============================] - 135s 12ms/sample - loss: 0.1463 - acc: 0.9585 - val_loss: 0.1767 - val_acc: 0.9462
Epoch 2/10
11706/11706 [==============================] - 129s 11ms/sample - loss: 0.1485 - acc: 0.9591 - val_loss: 0.1751 - val_acc: 0.9493
Epoch 3/10
11706/11706 [==============================] - 135s 12ms/sample - loss: 0.1355 - acc: 0.9614 - val_loss: 0.1683 - val_acc: 0.9450
Epoch 4/10
11706/11706 [==============================] - 126s 11ms/sample - loss: 0.1321 - acc: 0.9618 - val_loss: 0.1626 - val_acc: 0.9458
Epoch 5/10
11706/11706 [==============================] - 109s 9ms/sample - loss: 0.1283 - acc: 0.9639 - val_loss: 0.1593 - val_acc: 0.9523
Epoch 6/10
11706/11706 [==============================] - 121s 10ms/sample - loss: 0.1180 - acc: 0.9654 - val_loss: 0.1681 - val_acc: 0.9454
Epoch 7/10
11706/11706 [==============================] - 132s 11ms/sample - loss: 0.1159 - acc: 0.9651 - 

In [21]:
conv_model.save("./Checkpoints/conv_model2.h5")

In [22]:
conv_model.save_weights("./Checkpoints/conv_weights2")

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pred = full_model.predict([x_test_text,x_test_title])
pred = [np.argmax(x) for x in pred]

print(f"F1-score : {f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)}")
print(f"Accuracy-score : {accuracy_score([np.argmax(x) for x in y_test],pred)}")

print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")

F1-score : 0.9519756838905775
Accuracy-score : 0.9500947567909034
Precision : 0.9173989455184535
Recall  : 0.9892608970309539


In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

0.9500947567909033

In [29]:
from sklearn.metrics import accuracy_score

pred = conv_model.predict(x_test)
pred = [np.argmax(x) for x in pred]



In [30]:
from sklearn.metrics import f1_score
f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)

0.9362930077691454

In [31]:
accuracy_score([np.argmax(x) for x in y_test],pred)

0.9093779602147143

In [32]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

0.8797661587489404

In [34]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 0)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 0)}")

Precision : 0.8851894374282434
Recall  : 0.8048016701461378


In [1]:
import pickle
def save_pickle_file(file,file_path,protocol=None):
    with open(file_path,"wb") as f:
        if protocol:
            pickle.dump(file,f,protocol = protocol)
        else:
            pickle.dump(file,f)

def load_pickle_file(file_path, protocol=None):
     with open(file_path,"rb") as f:
        if protocol:
            return pickle.load(f)
        else:
            return pickle.load(f)
